In [1]:
#%matplotlib inline
import numpy as np
from scipy.stats import truncnorm
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

### Draw inclinations and jet angles and compare

In [2]:
# Draw N_bns times from uniform in cos(iota) and convert to iotas in degrees
N_bns = 1e6
iota_draws = np.rad2deg(np.arccos(np.random.uniform(1e-9, 1, N_bns)))

# Plot to check we are drawing enough times
#fig = plt.figure()
#ax = fig.gca()
#ax.hist(iota_draws, 50, facecolor='green', alpha=0.75)
#plt.show()

# Set the values for the opening angle mean (theta) and width (sigma) for
# which to perform calculation
thetas = 5 * (np.arange(6) + 1)
sigmas = np.arange(15) + 1
ratios = {}
for theta in thetas:
    ratio = []
    for sigma in sigmas:
        # Draw N_bns times from the normal distribution of mean theta,
        # width sigma, and truncated between 0 and 90
        a, b = (0. - theta) / sigma, (90. - theta) / sigma
        theta_draws = sigma * truncnorm.rvs(a, b, size=N_bns) + theta
        assert(max(theta_draws) <= 90)
        assert(min(theta_draws) > 0)
        N_grb = sum(iota_draws <= theta_draws)
        ratio.extend([float(N_grb) / float(N_bns)])
    
    ratios[theta] = np.array(ratio)

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()


#### Make Figure 1

In [4]:
fig = plt.figure()
ax = fig.gca()
markers = ['o', 'v', 's', '^', 'D', 'h']
lines = ['-', ':', '-.', '--', '-', ':']
for theta in thetas:
    y = ratios[theta] * 100
    ax.plot(sigmas, y, marker=markers.pop(), linestyle=lines.pop(),
            label=r'$\langle \theta \rangle = %d$' % theta)

x_major_locator = MultipleLocator(2)
x_major_formatter = FormatStrFormatter('%d')
x_minor_locator = MultipleLocator(1)
ax.xaxis.set_major_locator(x_major_locator)
ax.xaxis.set_major_formatter(x_major_formatter)
ax.xaxis.set_minor_locator(x_minor_locator)
y_major_locator = MultipleLocator(5)
y_major_formatter = FormatStrFormatter('%d')
y_minor_locator = MultipleLocator(1)
ax.yaxis.set_major_locator(y_major_locator)
ax.yaxis.set_major_formatter(y_major_formatter)
ax.yaxis.set_minor_locator(y_minor_locator)
ax.set_xlabel(r'$\sigma_{\theta} \quad (^{\circ})$', fontsize=18)
ax.set_ylabel(r'$N_{\mathrm{GRB}} / N_{\mathrm{BNS}} \quad (\%)$', fontsize=18)
ax.set_ylim([0, 20])
ax.grid()
ax.legend(loc='best', ncol=3)
plt.savefig('figure1.pdf')
plt.savefig('figure1.png')
plt.savefig('figure1.eps')
#plt.show()